In [20]:
# Standard packages
from skimage import morphology

import numpy as np
import pandas as pd

import plotly.graph_objects as go

import SimpleITK as sitk
import porespy as ps
from radiomics import featureextractor, shape, firstorder, glcm, glrlm

from collections import Counter

from multiprocessing import Pool

# Custom packages
import auxiliary.values as v
from filtering import cardiac_region as cr
from auxiliary.data.dataset_ht import HtDataset
from auxiliary.utils.colors import bcolors as c
from auxiliary.data import imaging

In [45]:
specimen = '0806_E6'
tissue = 'myocardium'

ds = HtDataset()
path_nu_feat = ds.read_features(specimen, 'Nuclei', tissue)
path_mem_feat = ds.read_features(specimen, 'Membrane', tissue)

path_nu_img, _ = ds.read_specimen(specimen, 'Nuclei', 'Segmentation', verbose=1)
path_mem_img, _ = ds.read_specimen(specimen, 'Membrane', 'Segmentation', verbose=1)
path_lines, _ = ds.read_line(specimen, verbose=1)

lines = imaging.read_image(path_lines, verbose=1)
nuclei = imaging.read_image(path_nu_img, verbose=1)
nuclei = cr.filter_by_tissue(nuclei, lines, tissue, dilate=1, verbose=1)

membrane = imaging.read_image(path_mem_img, verbose=1)
membrane = cr.filter_by_tissue(membrane, lines, tissue, dilate=1, verbose=1)

feat_nu = pd.read_csv(path_nu_feat)
feat_mem = pd.read_csv(path_mem_feat)

print(len(feat_nu), len(feat_mem))

	Found: 20190806_E6_mask_myocardium_splanchnic.nii.gz
	Found: 20190806_E6_mGFP_XYZ_predictions_GASP.nii.gz
	Found: line_20190806_E6.nii.gz
Reading NIfTI: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/Gr4/Segmentation/LinesTissue/line_20190806_E6.nii.gz
Reading NIfTI: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/Gr4/Segmentation/Nuclei/20190806_E6_mask_myocardium_splanchnic.nii.gz
Filtering image by tissue: myocardium...
Dilating mask: (3x3) 1 times...
[==================================================] 100.00%
Reading NIfTI: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/Gr4/Segmentation/Membrane/20190806_E6_mGFP_XYZ_predictions_GASP.nii.gz
Filtering image by tissue: myocardium...
Dilating mask: (3x3) 1 times...
[==================================================] 100.00%
2578 1656


Count number of nuclei inside each membrane

In [40]:
# feat_nu = pd.DataFrame(
#     columns=['x', 'y', 'z', 'volume', 'label', 'cell_in_props'],
#     data=[
#         [int(x), int(y), int(z), vol, label] 
#         for (x, y, z), vol, label
#         in zip(
#             list(map(
#                 lambda aux: aux.replace('[', '').replace(']', '').replace(',', ' ').split(), 
#                 nuclei.centroids
#             )),
#             nuclei.volumes,
#             nuclei.original_labels
#         )
#     ]
# )
# 
# feat_mem = pd.DataFrame(
#     columns=['x', 'y', 'z', 'volume', 'label', 'cell_in_props'],
#     data=[
#         [int(x), int(y), int(z), vol, label] 
#         for (x, y, z), vol, label
#         in zip(
#             list(map(
#                 lambda aux: aux.replace('[', '').replace(']', '').replace(',', ' ').split(), 
#                 membrane.centroids
#             )),
#             membrane.volumes,
#             membrane.original_labels
#         )
#     ]
# )